## Imports and read data

In [31]:
import pandas as pd
from helpers import read_and_describe, calculate_frequent_patterns, generate_association_rules
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 15)

import warnings
warnings.filterwarnings("ignore") # ignores warnings

df = read_and_describe('cleaned_data.csv')

Data types:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 527950 entries, 0 to 527949
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    527950 non-null  int64         
 1   StockCode    527950 non-null  object        
 2   Description  527950 non-null  object        
 3   Quantity     527950 non-null  int64         
 4   InvoiceDate  527950 non-null  datetime64[ns]
 5   UnitPrice    527950 non-null  float64       
 6   CustomerID   527950 non-null  object        
 7   Country      527950 non-null  object        
 8   TotalPrice   527950 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 36.3+ MB
None

Numeric data description:

           InvoiceNo       Quantity      UnitPrice     TotalPrice
count  527950.000000  527950.000000  527950.000000  527950.000000
mean   559987.737661       8.802667       3.277006      17.064975
std     13432.

### Frequent patterns (Itemsets)

In [2]:
fp_10 = calculate_frequent_patterns(df, min_support = 0.1)
fp_05 = calculate_frequent_patterns(df, min_support = 0.05)
fp_01 = calculate_frequent_patterns(df, min_support = 0.01)

print(f'Number of frequent patterns for support 10% = {fp_10.shape[0]}')
print(f'Number of frequent patterns for support 05% = {fp_05.shape[0]}')
print(f'Number of frequent patterns for support 01% = {fp_01.shape[0]}')

Number of frequent patterns for support 10% = 3
Number of frequent patterns for support 05% = 33
Number of frequent patterns for support 01% = 1874


In [18]:
fp_01

,support,itemsets
0,0.013592,( SET 2 TEA TOWELS I LOVE LONDON )
1,0.016068,(10 COLOUR SPACEBOY PEN)
2,0.012683,(12 MESSAGE CARDS WITH ENVELOPES)
3,0.017937,(12 PENCIL SMALL TUBE WOODLAND)
4,0.018291,(12 PENCILS SMALL TUBE RED RETROSPOT)
...,...,...
1869,0.011369,"(LUNCH BAG BLACK SKULL., LUNCH BAG CARS BLUE, LUNCH BAG RED RETROSPOT, LUNCH BAG PINK POLKADOT)"
1870,0.010459,"(LUNCH BAG BLACK SKULL., LUNCH BAG CARS BLUE, LUNCH BAG RED RETROSPOT, LUNCH BAG SUKI DESIGN )"
1871,0.010005,"(LUNCH BAG BLACK SKULL., LUNCH BAG SPACEBOY DESIGN , LUNCH BAG RED RETROSPOT, LUNCH BAG PINK POLKADOT)"
1872,0.010358,"(LUNCH BAG SUKI DESIGN , LUNCH BAG BLACK SKULL., LUNCH BAG RED RETROSPOT, LUNCH BAG PINK POLKADOT)"


#### After multiple tries, minimum support of 1% generated a reasonable number of frequent patterns given the large dataset

### Association rules

In [22]:
print("Association rule for min confidence 0.7 and min lift 1\n")
display(generate_association_rules(fp_01, 0.7, 1).reset_index())

print("Association rule for min confidence 0.7 and min lift 10\n")
display(generate_association_rules(fp_01, 0.7, 10).reset_index())

print("Association rule for min confidence 0.8 and min lift 1\n")
display(generate_association_rules(fp_01, 0.8, 1).reset_index())

print("Association rule for min confidence 0.8 and min lift 10\n")
display(generate_association_rules(fp_01, 0.8, 10).reset_index())

print("Association rule for min confidence 0.9 and min lift 1\n")
display(generate_association_rules(fp_01, 0.9, 1).reset_index())

print("Association rule for min confidence 0.9 and min lift 10\n")
display(generate_association_rules(fp_01, 0.9, 10).reset_index())

Association rule for min confidence 0.7 and min lift 1



,index,rule,support,confidence,lift
0,0,{'PAINTED METAL PEARS ASSORTED'} => {'ASSORTED COLOUR BIRD ORNAMENT'},0.013087,0.700000,9.521443
1,1,{'BAKING SET SPACEBOY DESIGN'} => {'BAKING SET 9 PIECE RETROSPOT '},0.016876,0.719828,15.252792
2,2,{'KITCHEN METAL SIGN'} => {'BATHROOM METAL SIGN'},0.011116,0.721311,22.623574
3,3,{'TOILET METAL SIGN'} => {'BATHROOM METAL SIGN'},0.014754,0.720988,22.613418
4,4,{'CANDLEHOLDER PINK HANGING HEART'} => {'WHITE HANGING HEART T-LIGHT HOLDER'},0.014249,0.708543,6.215766
...,...,...,...,...,...
260,260,"{'STRAWBERRY CHARLOTTE BAG', 'WOODLAND CHARLOTTE BAG', 'RED RETROSPOT CHARLOTTE BAG', 'CHARLOTTE BAG SUKI DESIGN'} => {'CHARLOTTE BAG PINK POLKADOT'}",0.010308,0.784615,20.927659
261,261,"{'STRAWBERRY CHARLOTTE BAG', 'WOODLAND CHARLOTTE BAG', 'CHARLOTTE BAG PINK POLKADOT', 'CHARLOTTE BAG SUKI DESIGN'} => {'RED RETROSPOT CHARLOTTE BAG'}",0.010308,0.918919,17.622407
262,262,"{'STRAWBERRY CHARLOTTE BAG', 'CHARLOTTE BAG PINK POLKADOT', 'RED RETROSPOT CHARLOTTE BAG', 'CHARLOTTE BAG SUKI DESIGN'} => {'WOODLAND CHARLOTTE BAG'}",0.010308,0.832653,19.782757
263,263,"{'STRAWBERRY CHARLOTTE BAG', 'WOODLAND CHARLOTTE BAG', 'CHARLOTTE BAG PINK POLKADOT'} => {'CHARLOTTE BAG SUKI DESIGN', 'RED RETROSPOT CHARLOTTE BAG'}",0.010308,0.775665,30.278489


Association rule for min confidence 0.7 and min lift 10



,index,rule,support,confidence,lift
0,1,{'BAKING SET SPACEBOY DESIGN'} => {'BAKING SET 9 PIECE RETROSPOT '},0.016876,0.719828,15.252792
1,2,{'KITCHEN METAL SIGN'} => {'BATHROOM METAL SIGN'},0.011116,0.721311,22.623574
2,3,{'TOILET METAL SIGN'} => {'BATHROOM METAL SIGN'},0.014754,0.720988,22.613418
3,5,{'CHARLOTTE BAG PINK POLKADOT'} => {'RED RETROSPOT CHARLOTTE BAG'},0.026376,0.703504,13.491326
4,6,{'CHILDRENS CUTLERY DOLLY GIRL '} => {'CHILDRENS CUTLERY SPACEBOY '},0.011167,0.759450,41.750773
...,...,...,...,...,...
189,260,"{'STRAWBERRY CHARLOTTE BAG', 'WOODLAND CHARLOTTE BAG', 'RED RETROSPOT CHARLOTTE BAG', 'CHARLOTTE BAG SUKI DESIGN'} => {'CHARLOTTE BAG PINK POLKADOT'}",0.010308,0.784615,20.927659
190,261,"{'STRAWBERRY CHARLOTTE BAG', 'WOODLAND CHARLOTTE BAG', 'CHARLOTTE BAG PINK POLKADOT', 'CHARLOTTE BAG SUKI DESIGN'} => {'RED RETROSPOT CHARLOTTE BAG'}",0.010308,0.918919,17.622407
191,262,"{'STRAWBERRY CHARLOTTE BAG', 'CHARLOTTE BAG PINK POLKADOT', 'RED RETROSPOT CHARLOTTE BAG', 'CHARLOTTE BAG SUKI DESIGN'} => {'WOODLAND CHARLOTTE BAG'}",0.010308,0.832653,19.782757
192,263,"{'STRAWBERRY CHARLOTTE BAG', 'WOODLAND CHARLOTTE BAG', 'CHARLOTTE BAG PINK POLKADOT'} => {'CHARLOTTE BAG SUKI DESIGN', 'RED RETROSPOT CHARLOTTE BAG'}",0.010308,0.775665,30.278489


Association rule for min confidence 0.8 and min lift 1



...

,index,rule,support,confidence,lift
0,0,{'PINK REGENCY TEACUP AND SAUCER'} => {'GREEN REGENCY TEACUP AND SAUCER'},0.031934,0.826144,16.140387
1,1,{'HERB MARKER BASIL'} => {'HERB MARKER MINT'},0.010510,0.859504,70.876860
2,2,{'HERB MARKER MINT'} => {'HERB MARKER BASIL'},0.010510,0.866667,70.876860
3,3,{'HERB MARKER BASIL'} => {'HERB MARKER PARSLEY'},0.010712,0.876033,72.846934
4,4,{'HERB MARKER PARSLEY'} => {'HERB MARKER BASIL'},0.010712,0.890756,72.846934
...,...,...,...,...,...
82,93,"{'PINK REGENCY TEACUP AND SAUCER', 'ROSES REGENCY TEACUP AND SAUCER ', 'REGENCY CAKESTAND 3 TIER'} => {'GREEN REGENCY TEACUP AND SAUCER'}",0.015209,0.909366,17.766292
83,101,"{'CHARLOTTE BAG PINK POLKADOT', 'WOODLAND CHARLOTTE BAG', 'RED RETROSPOT CHARLOTTE BAG', 'CHARLOTTE BAG SUKI DESIGN'} => {'STRAWBERRY CHARLOTTE BAG'}",0.010308,0.812749,22.278553
84,102,"{'STRAWBERRY CHARLOTTE BAG', 'CHARLOTTE BAG PINK POLKADOT', 'WOODLAND CHARLOTTE BAG', 'RED RETROSPOT CHARLOTTE BAG'} => {'CHARLOTTE BAG SUKI DESIGN'}",0.010308,0.842975,18.915331
85,103,"{'STRAWBERRY CHARLOTTE BAG', 'WOODLAND CHARLOTTE BAG', 'CHARLOTTE BAG PINK POLKADOT', 'CHARLOTTE BAG SUKI DESIGN'} => {'RED RETROSPOT CHARLOTTE BAG'}",0.010308,0.918919,17.622407


Association rule for min confidence 0.9 and min lift 1



,index,rule,support,confidence,lift
0,0,{'HERB MARKER PARSLEY'} => {'HERB MARKER ROSEMARY'},0.010864,0.903361,74.493435
1,1,{'HERB MARKER THYME'} => {'HERB MARKER ROSEMARY'},0.011167,0.932489,76.895411
2,2,{'HERB MARKER ROSEMARY'} => {'HERB MARKER THYME'},0.011167,0.920833,76.895411
3,3,{'REGENCY TEA PLATE PINK'} => {'REGENCY TEA PLATE GREEN '},0.014047,0.911475,47.346483
4,4,"{'PINK REGENCY TEACUP AND SAUCER', 'ROSES REGENCY TEACUP AND SAUCER '} => {'GREEN REGENCY TEACUP AND SAUCER'}",0.027336,0.904682,17.674795
5,5,"{'REGENCY TEA PLATE ROSES ', 'GREEN REGENCY TEACUP AND SAUCER'} => {'ROSES REGENCY TEACUP AND SAUCER '}",0.010257,0.906250,16.840933
6,6,"{'HERB MARKER BASIL', 'HERB MARKER THYME'} => {'HERB MARKER ROSEMARY'}",0.010106,0.952381,78.535714
7,7,"{'HERB MARKER BASIL', 'HERB MARKER ROSEMARY'} => {'HERB MARKER THYME'}",0.010106,0.934579,78.043298
8,8,"{'HERB MARKER THYME', 'HERB MARKER ROSEMARY'} => {'HERB MARKER BASIL'}",0.010106,0.904977,74.009947
9,9,"{'HERB MARKER THYME', 'HERB MARKER ROSEMARY'} => {'HERB MARKER PARSLEY'}",0.010257,0.918552,76.382619


Association rule for min confidence 0.9 and min lift 10



,index,rule,support,confidence,lift
0,0,{'HERB MARKER PARSLEY'} => {'HERB MARKER ROSEMARY'},0.010864,0.903361,74.493435
1,1,{'HERB MARKER THYME'} => {'HERB MARKER ROSEMARY'},0.011167,0.932489,76.895411
2,2,{'HERB MARKER ROSEMARY'} => {'HERB MARKER THYME'},0.011167,0.920833,76.895411
3,3,{'REGENCY TEA PLATE PINK'} => {'REGENCY TEA PLATE GREEN '},0.014047,0.911475,47.346483
4,4,"{'PINK REGENCY TEACUP AND SAUCER', 'ROSES REGENCY TEACUP AND SAUCER '} => {'GREEN REGENCY TEACUP AND SAUCER'}",0.027336,0.904682,17.674795
5,5,"{'REGENCY TEA PLATE ROSES ', 'GREEN REGENCY TEACUP AND SAUCER'} => {'ROSES REGENCY TEACUP AND SAUCER '}",0.010257,0.906250,16.840933
6,6,"{'HERB MARKER BASIL', 'HERB MARKER THYME'} => {'HERB MARKER ROSEMARY'}",0.010106,0.952381,78.535714
7,7,"{'HERB MARKER BASIL', 'HERB MARKER ROSEMARY'} => {'HERB MARKER THYME'}",0.010106,0.934579,78.043298
8,8,"{'HERB MARKER THYME', 'HERB MARKER ROSEMARY'} => {'HERB MARKER BASIL'}",0.010106,0.904977,74.009947
9,9,"{'HERB MARKER THYME', 'HERB MARKER ROSEMARY'} => {'HERB MARKER PARSLEY'}",0.010257,0.918552,76.382619


##### There are 265 good association rules for a minimum condifence of 0.7 and a lift higher than 1
##### There are 194 good association rules for a minimum condifence of 0.7 and a lift higher than 10 // Very high dependency
##### There are 105 very good association rules for a minimum condifence of 0.8 and a lift higher than 1
##### There are 87 very good association rules for a minimum condifence of 0.8 and a lift higher than 10 // Very high dependency
##### There are 19 excellent association rules for a minimum condifence of 0.9 and a lift higher than 10 // Very high dependency
##### People love to combine herb products together.
##### It's a worldwide data -> Trying the same per country using higher support value

# Frequent patterns for different countries

In [32]:
countries = ['EIRE','Belgium', 'France', 'Germany', 'Netherlands', 'Norway', 'Spain', 'Switzerland', 'Australia']
min_support_values = {
    'EIRE': 0.05, # Ireland 
    'Belgium': 0.05,
    'France': 0.04,
    'Germany': 0.025,
    'Netherlands': 0.08,
    'Norway': 0.1,
    'Spain': 0.05,
    'Switzerland': 0.1,
    'Australia': 0.07
}

for country in countries:
    min_confidence = 0.6
    # Display the country name
    display(HTML(f"<div style='font-size: 30px;'>{country}</div>"))

    # Generate and display the association rules
    frequent_patterns = calculate_frequent_patterns(df[df['Country'] == country], min_support_values.get(country))
    rules = generate_association_rules(frequent_patterns, min_confidence)
    display(rules)

,rule,support,confidence,lift
0,{'60 TEATIME FAIRY CAKE CASES'} => {'PACK OF 72 RETROSPOT CAKE CASES'},0.052817,0.625000,5.546875
1,{'GREEN REGENCY TEACUP AND SAUCER'} => {'PINK REGENCY TEACUP AND SAUCER'},0.091549,0.722222,7.325397
2,{'PINK REGENCY TEACUP AND SAUCER'} => {'GREEN REGENCY TEACUP AND SAUCER'},0.091549,0.928571,7.325397
3,{'GREEN REGENCY TEACUP AND SAUCER'} => {'REGENCY CAKESTAND 3 TIER'},0.088028,0.694444,2.817460
4,{'REGENCY MILK JUG PINK '} => {'GREEN REGENCY TEACUP AND SAUCER'},0.059859,0.680000,5.364444
...,...,...,...,...
196,"{'ROSES REGENCY TEACUP AND SAUCER ', 'REGENCY TEA PLATE ROSES '} => {'REGENCY CAKESTAND 3 TIER', 'REGENCY TEA PLATE GREEN '}",0.052817,0.789474,11.210526
197,"{'REGENCY CAKESTAND 3 TIER', 'REGENCY TEA PLATE GREEN '} => {'ROSES REGENCY TEACUP AND SAUCER ', 'REGENCY TEA PLATE ROSES '}",0.052817,0.750000,11.210526
198,"{'REGENCY CAKESTAND 3 TIER', 'REGENCY TEA PLATE ROSES '} => {'ROSES REGENCY TEACUP AND SAUCER ', 'REGENCY TEA PLATE GREEN '}",0.052817,0.750000,12.529412
199,"{'REGENCY TEA PLATE GREEN ', 'REGENCY TEA PLATE ROSES '} => {'ROSES REGENCY TEACUP AND SAUCER ', 'REGENCY CAKESTAND 3 TIER'}",0.052817,0.750000,6.656250


,rule,support,confidence,lift
0,{'DOLLY GIRL BABY GIFT SET'} => {' SPACEBOY BABY GIFT SET'},0.061224,0.857143,12.000000
1,{' SPACEBOY BABY GIFT SET'} => {'DOLLY GIRL BABY GIFT SET'},0.061224,0.857143,12.000000
2,{'PACK OF 60 MUSHROOM CAKE CASES'} => {'60 CAKE CASES DOLLY GIRL DESIGN'},0.051020,0.625000,6.805556
3,{'60 CAKE CASES DOLLY GIRL DESIGN'} => {'PACK OF 60 SPACEBOY CAKE CASES'},0.061224,0.666667,7.259259
4,{'PACK OF 60 SPACEBOY CAKE CASES'} => {'60 CAKE CASES DOLLY GIRL DESIGN'},0.061224,0.666667,7.259259
...,...,...,...,...
287,"{'PLASTERS IN TIN STRONGMAN', 'SPACEBOY LUNCH BOX ', 'ROUND SNACK BOXES SET OF4 WOODLAND '} => {'ROUND SNACK BOXES SET OF 4 FRUITS '}",0.051020,1.000000,4.454545
288,"{'ROUND SNACK BOXES SET OF 4 FRUITS ', 'SPACEBOY LUNCH BOX ', 'PLASTERS IN TIN STRONGMAN'} => {'ROUND SNACK BOXES SET OF4 WOODLAND '}",0.051020,1.000000,2.578947
289,"{'PLASTERS IN TIN STRONGMAN', 'ROUND SNACK BOXES SET OF4 WOODLAND '} => {'ROUND SNACK BOXES SET OF 4 FRUITS ', 'SPACEBOY LUNCH BOX '}",0.051020,0.714286,7.000000
290,"{'ROUND SNACK BOXES SET OF 4 FRUITS ', 'PLASTERS IN TIN STRONGMAN'} => {'SPACEBOY LUNCH BOX ', 'ROUND SNACK BOXES SET OF4 WOODLAND '}",0.051020,0.714286,4.666667


,rule,support,confidence,lift
0,{'ALARM CLOCK BAKELIKE GREEN'} => {'ALARM CLOCK BAKELIKE PINK'},0.075718,0.763158,7.307237
1,{'ALARM CLOCK BAKELIKE PINK'} => {'ALARM CLOCK BAKELIKE GREEN'},0.075718,0.725000,7.307237
2,{'ALARM CLOCK BAKELIKE GREEN'} => {'ALARM CLOCK BAKELIKE RED '},0.080940,0.815789,8.444523
3,{'ALARM CLOCK BAKELIKE RED '} => {'ALARM CLOCK BAKELIKE GREEN'},0.080940,0.837838,8.444523
4,{'ALARM CLOCK BAKELIKE PINK'} => {'ALARM CLOCK BAKELIKE RED '},0.075718,0.725000,7.504730
...,...,...,...,...
62,"{'SET/6 RED SPOTTY PAPER PLATES', 'SET/20 RED RETROSPOT PAPER NAPKINS '} => {'SET/6 RED SPOTTY PAPER CUPS'}",0.101828,0.975000,6.915278
63,"{'SET/6 RED SPOTTY PAPER CUPS', 'SET/20 RED RETROSPOT PAPER NAPKINS '} => {'SET/6 RED SPOTTY PAPER PLATES'}",0.101828,0.975000,7.468500
64,"{'SET/6 RED SPOTTY PAPER PLATES'} => {'SET/6 RED SPOTTY PAPER CUPS', 'SET/20 RED RETROSPOT PAPER NAPKINS '}",0.101828,0.780000,7.468500
65,"{'SET/6 RED SPOTTY PAPER CUPS'} => {'SET/6 RED SPOTTY PAPER PLATES', 'SET/20 RED RETROSPOT PAPER NAPKINS '}",0.101828,0.722222,6.915278


,rule,support,confidence,lift
0,{'SCANDINAVIAN REDS RIBBONS'} => {'6 RIBBONS RUSTIC CHARM'},0.029345,0.619048,5.834853
1,{'RED HARMONICA IN BOX '} => {'BLUE HARMONICA IN BOX '},0.029345,0.619048,12.465368
2,{'BLUE HARMONICA IN BOX '} => {'ROUND SNACK BOXES SET OF4 WOODLAND '},0.033860,0.681818,2.696834
3,{'BLUE POLKADOT BOWL'} => {'RED RETROSPOT CUP'},0.027088,0.800000,11.075000
4,{'PINK POLKADOT CUP'} => {'BLUE POLKADOT CUP'},0.027088,0.666667,13.424242
...,...,...,...,...
68,"{'WOODLAND CHARLOTTE BAG', 'ROUND SNACK BOXES SET OF4 WOODLAND '} => {'ROUND SNACK BOXES SET OF 4 FRUITS '}",0.040632,0.620690,3.818966
69,"{'ROUND SNACK BOXES SET OF 4 FRUITS ', 'PLASTERS IN TIN CIRCUS PARADE ', 'ROUND SNACK BOXES SET OF4 WOODLAND '} => {'PLASTERS IN TIN WOODLAND ANIMALS'}",0.027088,0.631579,4.441103
70,"{'PLASTERS IN TIN WOODLAND ANIMALS', 'ROUND SNACK BOXES SET OF 4 FRUITS ', 'ROUND SNACK BOXES SET OF4 WOODLAND '} => {'PLASTERS IN TIN CIRCUS PARADE '}",0.027088,0.631579,5.279047
71,"{'PLASTERS IN TIN WOODLAND ANIMALS', 'PLASTERS IN TIN CIRCUS PARADE ', 'ROUND SNACK BOXES SET OF4 WOODLAND '} => {'ROUND SNACK BOXES SET OF 4 FRUITS '}",0.027088,0.666667,4.101852


,rule,support,confidence,lift
0,{'72 SWEETHEART FAIRY CAKE CASES'} => {'DOLLY GIRL LUNCH BOX'},0.096774,0.900000,3.804545
1,{'72 SWEETHEART FAIRY CAKE CASES'} => {'SPACEBOY LUNCH BOX '},0.096774,0.900000,2.989286
2,{'CARD BIRTHDAY COWBOY'} => {'SPACEBOY BIRTHDAY CARD'},0.086022,0.888889,4.862745
3,{'CARD BIRTHDAY COWBOY'} => {'SPACEBOY LUNCH BOX '},0.086022,0.888889,2.952381
4,{'CARD DOLLY GIRL '} => {'ROUND SNACK BOXES SET OF4 WOODLAND '},0.096774,0.750000,2.790000
...,...,...,...,...
301,"{'SPACEBOY BIRTHDAY CARD', 'PLASTERS IN TIN SPACEBOY'} => {'LUNCH BAG RED RETROSPOT', 'ROUND SNACK BOXES SET OF4 WOODLAND '}",0.086022,0.888889,9.185185
302,"{'SPACEBOY BIRTHDAY CARD', 'ROUND SNACK BOXES SET OF4 WOODLAND '} => {'LUNCH BAG RED RETROSPOT', 'PLASTERS IN TIN SPACEBOY'}",0.086022,0.615385,7.153846
303,"{'PLASTERS IN TIN SPACEBOY', 'ROUND SNACK BOXES SET OF4 WOODLAND '} => {'LUNCH BAG RED RETROSPOT', 'SPACEBOY BIRTHDAY CARD'}",0.086022,0.666667,7.750000
304,"{'LUNCH BAG RED RETROSPOT'} => {'SPACEBOY BIRTHDAY CARD', 'PLASTERS IN TIN SPACEBOY', 'ROUND SNACK BOXES SET OF4 WOODLAND '}",0.086022,0.800000,8.266667


,rule,support,confidence,lift
0,{'BREAD BIN DINER STYLE IVORY'} => {'BLACK KITCHEN SCALES'},0.12500,0.800000,5.120000
1,{'BLACK KITCHEN SCALES'} => {'BREAD BIN DINER STYLE IVORY'},0.12500,0.800000,5.120000
2,{'IVORY KITCHEN SCALES'} => {'BLACK KITCHEN SCALES'},0.15625,0.714286,4.571429
3,{'BLACK KITCHEN SCALES'} => {'IVORY KITCHEN SCALES'},0.15625,1.000000,4.571429
4,{'BREAD BIN DINER STYLE IVORY'} => {'IVORY KITCHEN SCALES'},0.12500,0.800000,3.657143
...,...,...,...,...
314,"{'SMALL FOLDING SCISSOR(POINTED EDGE)'} => {'GROW A FLYTRAP OR SUNFLOWER IN TIN', 'TRAVEL CARD WALLET PANTRY', 'CHOCOLATE HOT WATER BOTTLE', 'FENG SHUI PILLAR CANDLE'}",0.12500,1.000000,8.000000
315,"{'CHOCOLATE HOT WATER BOTTLE'} => {'FENG SHUI PILLAR CANDLE', 'GROW A FLYTRAP OR SUNFLOWER IN TIN', 'TRAVEL CARD WALLET PANTRY', 'SMALL FOLDING SCISSOR(POINTED EDGE)'}",0.12500,1.000000,8.000000
316,"{'FENG SHUI PILLAR CANDLE'} => {'GROW A FLYTRAP OR SUNFLOWER IN TIN', 'TRAVEL CARD WALLET PANTRY', 'CHOCOLATE HOT WATER BOTTLE', 'SMALL FOLDING SCISSOR(POINTED EDGE)'}",0.12500,1.000000,8.000000
317,"{'TRAVEL CARD WALLET PANTRY'} => {'FENG SHUI PILLAR CANDLE', 'GROW A FLYTRAP OR SUNFLOWER IN TIN', 'CHOCOLATE HOT WATER BOTTLE', 'SMALL FOLDING SCISSOR(POINTED EDGE)'}",0.12500,1.000000,8.000000


,rule,support,confidence,lift
0,{'ASSORTED COLOUR BIRD ORNAMENT'} => {'6 RIBBONS RUSTIC CHARM'},0.102273,0.750000,4.400000
1,{'6 RIBBONS RUSTIC CHARM'} => {'ASSORTED COLOUR BIRD ORNAMENT'},0.102273,0.600000,4.400000
2,{'PLASTERS IN TIN SKULLS'} => {'6 RIBBONS RUSTIC CHARM'},0.068182,0.600000,3.520000
3,{'PLASTERS IN TIN WOODLAND ANIMALS'} => {'6 RIBBONS RUSTIC CHARM'},0.056818,0.625000,3.666667
4,{'RED RETROSPOT CAKE STAND'} => {'6 RIBBONS RUSTIC CHARM'},0.056818,0.625000,3.666667
...,...,...,...,...
55,"{""POPPY'S PLAYHOUSE LIVINGROOM "", ""POPPY'S PLAYHOUSE BEDROOM ""} => {""POPPY'S PLAYHOUSE KITCHEN""}",0.056818,1.000000,12.571429
56,"{""POPPY'S PLAYHOUSE KITCHEN"", ""POPPY'S PLAYHOUSE BEDROOM ""} => {""POPPY'S PLAYHOUSE LIVINGROOM ""}",0.056818,0.714286,12.571429
57,"{""POPPY'S PLAYHOUSE LIVINGROOM ""} => {""POPPY'S PLAYHOUSE KITCHEN"", ""POPPY'S PLAYHOUSE BEDROOM ""}",0.056818,1.000000,12.571429
58,"{""POPPY'S PLAYHOUSE KITCHEN""} => {""POPPY'S PLAYHOUSE LIVINGROOM "", ""POPPY'S PLAYHOUSE BEDROOM ""}",0.056818,0.714286,12.571429


,rule,support,confidence,lift
0,{' SPACEBOY BABY GIFT SET'} => {'PLASTERS IN TIN WOODLAND ANIMALS'},0.10,1.000000,2.941176
1,{'RED RETROSPOT BOWL'} => {'BLUE POLKADOT BOWL'},0.12,0.750000,6.250000
2,{'BLUE POLKADOT BOWL'} => {'RED RETROSPOT BOWL'},0.12,1.000000,6.250000
3,{'BLUE POLKADOT BOWL'} => {'SPOTTY BUNTING'},0.10,0.833333,4.166667
4,{'CARD CIRCUS PARADE'} => {'SPACEBOY BIRTHDAY CARD'},0.10,1.000000,4.545455
...,...,...,...,...
219,"{'RED TOADSTOOL LED NIGHT LIGHT', 'ROUND SNACK BOXES SET OF4 WOODLAND '} => {'WATERING CAN BLUE ELEPHANT', 'WATERING CAN PINK BUNNY'}",0.10,0.625000,5.208333
220,"{'WATERING CAN PINK BUNNY', 'ROUND SNACK BOXES SET OF4 WOODLAND '} => {'WATERING CAN BLUE ELEPHANT', 'RED TOADSTOOL LED NIGHT LIGHT'}",0.10,1.000000,10.000000
221,"{'WATERING CAN BLUE ELEPHANT'} => {'RED TOADSTOOL LED NIGHT LIGHT', 'WATERING CAN PINK BUNNY', 'ROUND SNACK BOXES SET OF4 WOODLAND '}",0.10,0.714286,7.142857
222,"{'RED TOADSTOOL LED NIGHT LIGHT'} => {'WATERING CAN BLUE ELEPHANT', 'WATERING CAN PINK BUNNY', 'ROUND SNACK BOXES SET OF4 WOODLAND '}",0.10,0.625000,6.250000


,rule,support,confidence,lift
0,{'RED RETROSPOT CAKE STAND'} => {'36 PENCILS TUBE RED RETROSPOT'},0.071429,1.000000,14.000000
1,{'36 PENCILS TUBE RED RETROSPOT'} => {'RED RETROSPOT CAKE STAND'},0.071429,1.000000,14.000000
2,{'36 PENCILS TUBE RED RETROSPOT'} => {'SET OF 3 CAKE TINS PANTRY DESIGN '},0.071429,1.000000,6.222222
3,{'SET OF 6 SOLDIER SKITTLES'} => {'4 TRADITIONAL SPINNING TOPS'},0.071429,0.666667,7.466667
4,{'4 TRADITIONAL SPINNING TOPS'} => {'SET OF 6 SOLDIER SKITTLES'},0.071429,0.800000,7.466667
...,...,...,...,...
738,"{'HOMEMADE JAM SCENTED CANDLES'} => {'ROSES REGENCY TEACUP AND SAUCER ', 'SPACEBOY LUNCH BOX ', 'CIRCUS PARADE LUNCH BOX ', 'DOLLY GIRL LUNCH BOX', 'REGENCY CAKESTAND 3 TIER'}",0.071429,0.800000,11.200000
739,"{'DOLLY GIRL LUNCH BOX'} => {'ROSES REGENCY TEACUP AND SAUCER ', 'SPACEBOY LUNCH BOX ', 'HOMEMADE JAM SCENTED CANDLES', 'CIRCUS PARADE LUNCH BOX ', 'REGENCY CAKESTAND 3 TIER'}",0.071429,0.666667,9.333333
740,"{'SPACEBOY LUNCH BOX '} => {'ROSES REGENCY TEACUP AND SAUCER ', 'HOMEMADE JAM SCENTED CANDLES', 'CIRCUS PARADE LUNCH BOX ', 'DOLLY GIRL LUNCH BOX', 'REGENCY CAKESTAND 3 TIER'}",0.071429,0.666667,9.333333
741,"{'CIRCUS PARADE LUNCH BOX '} => {'ROSES REGENCY TEACUP AND SAUCER ', 'SPACEBOY LUNCH BOX ', 'HOMEMADE JAM SCENTED CANDLES', 'DOLLY GIRL LUNCH BOX', 'REGENCY CAKESTAND 3 TIER'}",0.071429,0.800000,11.200000


#### The table above shows a set of association rules per country
#### Each country minimum support has been chosen after try and error mainly to generate a reasonable ammount of itemsets (Not very low, not huge)
#### Minimum confidence is 0.6
#### Minimum lift is 1 to make sure there is no negative dependancy 